In [60]:
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html
# Import Libraries
import numpy as np 
import cv2 
import glob

In [34]:
# Load images from dataset 
filenames = glob.glob("AE4317_2019_datasets/calibration_frontcam/20190121-163447/*.jpg")
filenames.sort()
images = [cv2.imread(img) for img in filenames]
goodImg = []   


In [55]:
# Setup for Calibration
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = 340.15*np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

for img in images:
    img = cv2.rotate(img, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        goodImg.append(img)
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,6), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)
        # will show the image in a window 
#     cv2.imshow('image', img) 
#     k = cv2.waitKey(0) & 0xFF
#     # wait for ESC key to exit 
#     if k == 27:  
#         cv2.destroyAllWindows() 
cv2.destroyAllWindows()

In [65]:
#Calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [66]:
# Re-projection Error
mean_error = 0
tot_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error

print("total error: ", mean_error/len(objpoints))

total error:  0.0


In [56]:
# refine the camera matrix based on a free scaling parameter using cv2.getOptimalNewCameraMatrix()
img = images[40]#goodImg[5] #Choose a random Image from the dataset
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# Undistort Method 1
dst1 = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst1[y:y+h, x:x+w]
cv2.imwrite('calibresult.png', dst)

# Check the Results
cv2.imshow('Distroted', img)
cv2.imshow('Undistorted', dst1)
cv2.imshow('Undistroted & Cut', dst)

k = cv2.waitKey(0) & 0xFF
if k == 27:
    cv2.destroyAllWindows()

In [57]:
# Undistort Method 2
mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst1 = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst1[y:y+h, x:x+w]
cv2.imwrite('calibresult.png', dst)

# Check the Results
cv2.imshow('Distroted', img)
cv2.imshow('Undistorted', dst1)
cv2.imshow('Undistroted & Cut', dst)

k = cv2.waitKey(0) & 0xFF
if k == 27:
    cv2.destroyAllWindows()

total error:  0.0
